In [3]:
# import all packages
 
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler

In [4]:
def load_data():
    # load data
    data = np.loadtxt('DMDdata200.csv', delimiter=',')
    data = data[:, 1:]
    avg = np.mean(data)
    data = data - avg
    scaler = MinMaxScaler()
    X_one_column = data.reshape([-1,1])
    result_one_column = scaler.fit_transform(X_one_column)
    result = result_one_column.reshape(data.shape)
    x1 = result[:,:-1]
    x2 = result[:,1:]
    
    return x1, x2, data	

In [5]:
x1, x2, data = load_data()

In [6]:
p_rows = 10000

rand_int = []
for i in range(p_rows):
    rand_int.append(np.random.randint(0, x1.shape[0])) 

In [7]:
y1 = []
for i in range(len(rand_int)):
    y1.append(x1[i, :])

In [8]:
y2 = []
for i in range(len(rand_int)):
    y2.append(x2[i, :])

In [9]:
y1 = np.array(y1)
y2 = np.array(y2)

In [10]:
data_shape = (90824, p_rows)
percent_Trunc = 0.99
dt = 2.5e-05

In [11]:
# Compression matrix generation

C_gaussian = np.random.normal(0, 1, size=data_shape)

In [12]:
# compress data

y1 = np.dot(C_gaussian.T, x1)
y2 = np.dot(C_gaussian.T, x2)

In [13]:
#cDMD

startc = 0
endc = 0

startc = time.time()

[Uc, Sc, Vc] = np.linalg.svd(y1, full_matrices=False, compute_uv=True, hermitian=False)

menergyc = 0.0
rc = 0

for i in range(0, np.size(Sc)):
    menergyc += Sc[i]/np.sum(Sc)
    if menergyc > percent_Trunc:
        break
    rc = i
    
menergyc, rc

Atildec = Uc[:, :rc].T @ y2 @ Vc[:rc, :].T * np.reciprocal(Sc[: rc])
[eivValc,eivVecc] = np.linalg.eig(Atildec)

phic = x2 @ Vc[:rc, :].T @ np.diag(np.reciprocal(Sc[: rc])) @ eivVecc
phic = phic / np.linalg.norm(phic, axis=0)

bc = np.linalg.lstsq(phic, x1[:, 0])
S = np.diag(Sc[: rc])
num_modesc = phic.shape[1]
omegac = np.log(eivValc) / dt

endc = time.time()

print("Time taken for cDMD: ", endc - startc)

Time taken for cDMD:  9.832293272018433


C:\Users\sumuk\AppData\Local\Temp\ipykernel_14328\3659512206.py:27: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  bc = np.linalg.lstsq(phic, x1[:, 0])


In [14]:
# DMD

start = 0
end = 0

start = time.time()

[U, S, V] = np.linalg.svd(x1, full_matrices=False, compute_uv=True, hermitian=False)

menergy = 0.0
r = 0

for i in range(0, np.size(S)):
    menergy += S[i]/np.sum(S)
    if menergy > percent_Trunc:
        break
    r = i
    
menergy, r

Atilde = U[:, :r].T @ x2 @ V[:, :r] * np.reciprocal(S[: r])
[eivVal,eivVec] = np.linalg.eig(Atilde)

phi = x2 @ V[:r, :].T @ np.diag(np.reciprocal(S[: r])) @ eivVec
phi = phi / np.linalg.norm(phi, axis=0)
b = np.linalg.lstsq(phi, x1[:, 0])
S = np.diag(S[: r])
num_modes = phi.shape[1]
omega = np.log(eivVal) / dt

end = time.time()

print("Time taken for DMD: ", end - start)

Time taken for DMD:  105.60917139053345


C:\Users\sumuk\AppData\Local\Temp\ipykernel_14328\1636692992.py:26: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  b = np.linalg.lstsq(phi, x1[:, 0])


In [17]:
## Reconstruction of the data

def recon(phi, b, omega, num_steps, num_modes, dt):

    time = np.arange(num_steps) * dt
    time_dynamics = np.zeros((90824, num_steps), dtype=np.complex128)
    for i in range(num_steps):
            for j in range(num_modes):
                time_dynamics[:, i] = b[j] * phi[:, j] * np.exp(omega[j].imag * time[i])

    return time_dynamics

In [18]:
reconstructed_fieldc = recon(phic, bc[0], omegac, 2000, num_modesc, dt)
reconstructed_field = recon(phi, b[0], omega, 2000, num_modes, dt)

C:\Users\sumuk\AppData\Local\Temp\ipykernel_14328\152581265.py:9: RuntimeWarning: overflow encountered in exp
  time_dynamics[:, i] = b[j] * phi[:, j] * np.exp(omega[j].imag * time[i])


In [19]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmsc = sqrt(mean_squared_error(reconstructed_fieldc[:, :1999].real, x2[:, :]))
rms = sqrt(mean_squared_error(x2[:, :], reconstructed_field[:, :1999].real))
rmsc, rms

(0.5069136383017739, 0.49039389883163015)

In [ ]:
# from matplotlib import pyplot as plt

# x = [5000, 10000, 20000, 30000, 40000, 50000]
# t_p = [9.74, 11.73, 14.93, 16.83, 20.81, 24.2]
# t_g = [5.7, 7.5, 12.8, 19.5, 26.03, 31.3]

# err_g = [0.44, 0.44, 0.57, 0.5, 0.57, 0.57]

# err_p = [0.58, 0.57, 0.56, 0.51, 0.49, 0.49]

# plt.title("Time taken for cDMD - Acoustics")
# plt.xlabel("Number of rows")
# plt.ylabel("Time (s)")

# plt.plot(x, t_p, label = "p random rows")
# plt.plot(x, t_g, label = "Gaussian")
# plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)
# plt.legend()
# plt.savefig("time.png", dpi=300)

In [ ]:
# plt.title("Error for cDMD - Acoustics")
# plt.xlabel("Number of rows")
# plt.ylabel("Error")

# plt.plot(x, err_p, label = "p random rows")
# plt.plot(x, err_g, label = "Gaussian")
# plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)
# plt.legend()

# plt.savefig("error.png", dpi=300)